In [25]:
#Benötigte Packages
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import sqlite3

LoadExtensionError: Failed to load SpatiaLite extension. Verify that your python module sqlite3 has load_extension support and check that libspatialite is installed. Tried extension names: mod_spatialite, mod_spatialite.so, mod_spatialite.dylib

In [3]:
# Grundeinstellungen

# bestandteile der URL
## alle Zahlen in Grad

lons = [13.6]  # linker Rand
lats = [51.0]  # unterer Rand
zellweite_Lon = 0.1  # pro 0.1 ist die Zelle ca 7km breiter/schmaler
zellweite_Lat = 0.1  # pro 0.1 ist die Zelle ca 11km höher/niedriger
url1 = 'https://api.openstreetmap.org/api/0.6/trackpoints?bbox='  # bestandteile URL
url2 = '&page='  # bestandteile URL

In [ ]:
# BBOX und URL
# Schleife für das erstellen der Dateien und hinzufügen der Punkte zu einem df
for lon in lons:
    for lat in lats:
        bbox = f"{lon},{lat},{lon + zellweite_Lon},{lat + zellweite_Lat}"  # erstellen der BBOX
        page = 0
        os.makedirs("osmtracks", exist_ok=True)  # Ordner erstellen um die .gpx zu speichern
        while True:
            url = f"{url1}{bbox}{url2}{page}"
            response = requests.get(url)
            filestring = f"osmtracks/tracks_{page}.gpx"  # Dateipfad und Name
            with open(filestring, "wb") as file:
                file.write(response.content)
            
            # datei direkt in xml umwandeln
            new_files = filestring.replace(".gpx", ".xml")
            os.rename(filestring, new_files)
    
            page += 1  # Page um eins erhöhen
            
            size = os.path.getsize(new_files)  # Dateigröße für Abbruchbedingung
            # Abbruchbedingung sehr wichtig, sonst wiederholt sich die letzte Page unendlich oft, bessere Alternative als
            # Dateigröße finden
            if size < 500000:
                break


In [11]:
# Benötigte Dataframes usw.
tracks_df = pd.DataFrame(columns=["lat", "lon", "time", "trk_id", "pt_id", "name"])
track_df = pd.DataFrame(columns=["lat", "lon", "time", "trk_id", "pt_id", "name"])

In [ ]:
files = []

for file in os.listdir("/Forschungsprojekt/G613/osmtracks"):
    if file.endswith(".xml"):
        files.append(file)

trk_counter = 0
for file in files:
    tree = ET.parse("/Forschungsprojekt/G613/osmtracks/" + file)
    root = tree.getroot()
    for trk in root.findall(".//{http://www.topografix.com/GPX/1/0}trk"):
        for trkseg in trk.findall(".//{http://www.topografix.com/GPX/1/0}trkseg"):
            trk_counter += 1
            pt_id = 0
            for trkpt in trkseg.findall(".//{http://www.topografix.com/GPX/1/0}trkpt"):
                lat = trkpt.get("lat")
                lon = trkpt.get("lon")
                time = trkpt.find(".//{http://www.topografix.com/GPX/1/0}time").text if trkpt.find(".//{http://www.topografix.com/GPX/1/0}time") is not None else "NA"
                name = trkpt.find(".//{http://www.topografix.com/GPX/1/0}name").text if trkpt.find(".//{http://www.topografix.com/GPX/1/0}name") is not None else "NoName"
                pt_id += 1
                new_row = [lat, lon, time, trk_counter, pt_id, name]
                track_df.loc[len(track_df)] = new_row

    tracks_df = pd.concat([tracks_df, track_df], ignore_index=True)
    track_df = pd.DataFrame(columns=["lat", "lon", "time", "trk_id", "pt_id", "name"])


In [26]:
#verbindung aufbauen
conn = sqlite3.connect('forschungsprojekt.db')
conn.execute("DROP TABLE IF EXISTS forschungsprojekt")
conn.execute("CREATE TABLE forschungsprojekt (lat float8,lon float8,time time,trk_id bigint,pt_id bigint,name VARCHAR(255))")

In [29]:
#cur.execute("INSERT INTO forschungsprjekt.daten VALUES(%2)")
tracks_df.to_sql("daten",conn,"forschungsprojekt", if_exists="replace")
conn.commit()

         index         lat         lon                  time  trk_id  pt_id  \
0            0  51.0999913  13.6979035  2023-06-04T13:32:03Z       1      1   
1            1  51.0998256  13.6980003  2023-06-04T13:32:06Z       1      2   
2            2  51.0995470  13.6981006  2023-06-04T13:32:12Z       1      3   
3            3  51.0992351  13.6981376  2023-06-04T13:32:19Z       1      4   
4            4  51.0991044  13.6981409  2023-06-04T13:32:22Z       1      5   
...        ...         ...         ...                   ...     ...    ...   
204650  204650  51.0388694  13.6692057  2014-10-18T22:39:35Z     407   4996   
204651  204651  51.0388694  13.6692057  2014-10-18T22:39:37Z     407   4997   
204652  204652  51.0388694  13.6692057  2014-10-18T22:39:38Z     407   4998   
204653  204653  51.0388694  13.6692057  2014-10-18T22:39:40Z     407   4999   
204654  204654  51.0388694  13.6692058  2014-10-18T22:39:41Z     407   5000   

          name  
0       NoName  
1       NoName  


In [23]:
# Verbidnung schließen
conn.close()